# Optimizing corner detection

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os

from ipywidgets import interact

In [10]:
# we need to access the points of the contour in a circular manner
def get_circular(contour, index):
    """Gets the point at the index of the contour, 
    but wraps around if the index is out of bounds

    Args:
        contour (np.array(len, 2)): array of points
        index (int): index of the point to get

    Returns:
        np.array(2): the point
    """
    # make sure index is positive
    index = index % len(contour) + len(contour)
    return contour[index % len(contour)]

def get_slice_circular(contour, start, end):
    """Gets the slice of the contour from start to end, but
    wraps around if the index is out of bounds

    Args:
        contour (np.array(len, 2)): array of points
        start (int): index of the start of the slice, may be negative
        end (int): index of the end of the slice, may be negative

    Returns:
        np.array(end-start, 2): array of points that make up the slice
    """
    
    # make sure start and end are positive
    start = start % len(contour)
    end = end % len(contour)
    if start < 0:
        start += len(contour)
    if end < 0:
        end += len(contour)
    
    # make sure end is larger than start
    if end < start:
        end += len(contour)
    
    if end > len(contour):
        return np.concatenate((contour[start:], contour[:end - len(contour)]))
    
    return contour[start:end]

def get_contour(puzzle_piece_img):
    """gets the contour of the puzzle piece

    Args:
        puzzle_piece_img (Mat): image of the puzzle piece

    Returns:
        np.array(len, 2): list of points that make up the contour
    """
    img_gray = cv2.cvtColor(puzzle_piece_img, cv2.COLOR_BGR2GRAY)
    # Threshold the image
    _, img_thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    return contours[0][:, 0, :]

In [24]:
# create a list of all the images in the folder

# Gets the puzzle piece image
def get_piece_image(path):
    # images are of the format piece#.png
    img = cv2.imread(path)
    return img

piece_dir = '../dataset/pieces-clean'
image_files = os.listdir(piece_dir)
image_files = sorted(image_files, key=lambda x: int(x[5:-4]))

# load all off the images
piece_images = []
for i in range(len(image_files)):
    piece_images.append( get_piece_image(os.path.join(piece_dir, image_files[i])) )
    
@interact(index=(0, len(piece_images) - 1))
def view_piece(index):
    plt.imshow(piece_images[index])
    plt.show()

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

In [25]:
piece_contours = []
for i in range(len(piece_images)):
    piece_contours.append(get_contour(piece_images[i]))

@interact(index=(0, len(piece_contours) - 1))
def visualize_contour(index):
    plt.plot(piece_contours[index][:, 0], piece_contours[index][:, 1])
    plt.show()

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

Testing different corner indicators and parameters to find the best combination.

In [13]:
# angle indicator
def get_angle_indicators(contour, window_size=5):
    """Gets the angle indicators for the puzzle piece. The angle indicators are the
    angles between vectors along the contour of the puzzle piece.

    Returns:
        np.array(len(contour)): array of angles
    """
    
    angle_indicators = np.zeros(len(contour))
    for i in range(len(contour)):
        point = get_circular(contour, i)
        start  = get_circular(contour, i - window_size)
        end = get_circular(contour, i + window_size)
        
        vec_start = point - start
        vec_end = end - point
        angle_indicators[i] = np.arctan2(np.cross(vec_end, vec_start), np.dot(vec_end, vec_start))
    
    return angle_indicators

# test the angle indicator
for i in range(len(piece_contours)):
    try:
        get_angle_indicators(piece_contours[i])
    except:
        print(i)
    
@interact(index=(0, len(piece_images) - 1))
def view_angle_indicators(index):
    plt.plot(get_angle_indicators(piece_contours[index]))
    plt.show()

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

In [26]:
# center indicator
def get_center_indicators(contour, window_size=5):
    """Gets the center indicators for the puzzle piece. The center indicators indicate how
    well a corner contains the center of the puzzle piece. The center is defined as the point (200, 200)

    Returns:
        np.array(len(contour)): array of distances
    """
    
    center = np.array([200, 200])
    center_indicators = np.zeros(len(contour))
    for i in range(len(contour)):
        point = get_circular(contour, i)
        start  = get_circular(contour, i - window_size)
        end = get_circular(contour, i + window_size)
        
        vec_start = point - start
        vec_end = end - point
        vec_center = center - point
        
        # rotate vec_start by 135 degrees clockwise and vec_end by 45 degrees clockwise
        theta = -np.pi * 3 / 4
        rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
        vec_start = np.dot(rot, vec_start)
        theta = -np.pi / 4
        rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
        vec_end = np.dot(rot, vec_end)
        
        assert np.linalg.norm(vec_start) > 0
        assert np.linalg.norm(vec_end) > 0
        assert np.linalg.norm(vec_center) > 0
        
        center_indicators[i] = (np.dot(vec_start, vec_center) / np.linalg.norm(vec_start) + np.dot(vec_end, vec_center) / np.linalg.norm(vec_end)) / np.linalg.norm(vec_center)

    return center_indicators

# test the center indicator
for i in range(len(piece_contours)):
    try:
        get_center_indicators(piece_contours[i])
    except:
        print(i)

@interact(index=(0, len(piece_images) - 1))
def view_center_indicators(index):
    plt.plot(get_center_indicators(piece_contours[index]))
    plt.show()

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

In [15]:
# distance indicator
def get_distance_indicators(contour, window_size=5):
    """Gets the distance indicators for a contour. Tge distance indicators indicate how
    much further away the corner is from the center of the puzzle piece compared to its
    neighbors. The center is assumed to be the point (200, 200)

    Args:
        contour (_type_): _description_
        window_size (int, optional): _description_. Defaults to 5.
    """
    
    center = np.array([200, 200])
    distance_indicators = np.zeros(len(contour))
    for i in range(len(contour)):
        point = get_circular(contour, i)
        start  = get_circular(contour, i - window_size)
        end = get_circular(contour, i + window_size)
        
        distance = np.linalg.norm(point - center)
        
        distance_indicators[i] = 2*distance - np.linalg.norm(start - center) - np.linalg.norm(end - center)
    
    return distance_indicators

# test the distance indicator
for i in range(len(piece_contours)):
    try:
        get_distance_indicators(piece_contours[i])
    except:
        print(i)

@interact(index=(0, len(piece_images) - 1))
def view_distance_indicators(index):
    plt.plot(get_distance_indicators(piece_contours[index]))
    plt.show()

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

In [16]:
# show all indicators
@interact(index=(0, len(piece_images) - 1))
def view_all_indicators(index):
    plt.plot(get_angle_indicators(piece_contours[index]), label='angle')
    plt.plot(get_center_indicators(piece_contours[index])/2, label='center')
    plt.plot(get_distance_indicators(piece_contours[index])/8, label='distance')
    plt.legend()

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

In [18]:
# show all indicators
@interact(index=(0, len(piece_images) - 1))
def view_all_indicators(index, w1=1.0, w2=0.5, w3=0.2):
    angle_indicators = get_angle_indicators(piece_contours[index]) * w1
    center_indicators = get_center_indicators(piece_contours[index]) * w2
    distance_indicators = get_distance_indicators(piece_contours[index]) * w3
    plt.plot(angle_indicators + center_indicators + distance_indicators, label='total')

interactive(children=(IntSlider(value=499, description='index', max=999), FloatSlider(value=1.0, description='…

Find out which corner indicator is the best

In [19]:

def find_corners(contour, corner_size=40):
    
    angle_indicators = get_angle_indicators(contour)
    center_indicators = get_center_indicators(contour) * 0.5
    distance_indicators = get_distance_indicators(contour) * 0.125
    
    indicators = angle_indicators + center_indicators + distance_indicators
    
    p_corners = [0]
    for i in range(len(indicators)):
        if i - p_corners[-1] < corner_size:
            
            if i - p_corners[0] > len(indicators) - corner_size:
                # point is too close to first and last corner
                if indicators[i] > indicators[p_corners[-1]] and indicators[i] > indicators[p_corners[0]]:
                    # remove first corner
                    p_corners = p_corners[1:]
                    
                    p_corners[-1] = i
            
            # point is too close to the previous corner
            elif indicators[i] > indicators[p_corners[-1]]:
                p_corners[-1] = i
        
        elif i - p_corners[0] > len(indicators) - corner_size:
            
            # point is too close to the first corner
            if indicators[i] > indicators[p_corners[0]]:
                # remove first corner
                p_corners = p_corners[1:]
                
                p_corners.append(i)
        else:
            p_corners.append(i)
    
    # find the 4 best corners
    p_corners = np.array(p_corners)
    return np.sort(p_corners[np.argsort(indicators[p_corners])[:-5:-1]])

In [27]:
corners = []
for i in range(len(piece_contours)):
    corners.append(find_corners(piece_contours[i]))

# show all indicators
@interact(index=(0, len(piece_images) - 1))
def view_all_indicators(index):
    corner = find_corners(piece_contours[index])
    angle_indicators = get_angle_indicators(piece_contours[index])
    center_indicators = get_center_indicators(piece_contours[index]) * 3
    distance_indicators = get_distance_indicators(piece_contours[index]) * 0.5
    
    for i in range(len(corner)):
        plt.axvline(corner[i], color='r')
    plt.plot(angle_indicators + center_indicators + distance_indicators, label='total')

interactive(children=(IntSlider(value=499, description='index', max=999), Output()), _dom_classes=('widget-int…

In [28]:
def get_shortest_edge(corners, contour_len):
    shortest_edge = np.inf
    for i in range(4):
        length = corners[i] - corners[i-1]
        if length < 0:
            length += contour_len
        if length < shortest_edge:
            shortest_edge = length
    return shortest_edge
def get_longest_edge(corners, contour_len):
    longest_edge = 0
    for i in range(4):
        length = corners[i] - corners[i-1]
        if length < 0:
            length += contour_len
        if length > longest_edge:
            longest_edge = length
    return longest_edge

# find shortest and longest edges
short_lengths = np.zeros(len(piece_contours))
long_lengths = np.zeros(len(piece_contours))
for i in range(len(corners)):
    short_lengths[i] = get_shortest_edge(corners[i], len(piece_contours[i]))
    long_lengths[i] = get_longest_edge(corners[i], len(piece_contours[i]))

indices = np.argsort(short_lengths)
for i in range(10):
    print("Length: {}, Index: {}".format(short_lengths[indices[i]], indices[i]))

indices = np.argsort(long_lengths)
for i in range(10):
    print("Length: {}, Index: {}".format(long_lengths[indices[i]], indices[i]))

Length: 93.0, Index: 78
Length: 96.0, Index: 68
Length: 97.0, Index: 26
Length: 97.0, Index: 108
Length: 98.0, Index: 101
Length: 98.0, Index: 123
Length: 99.0, Index: 98
Length: 99.0, Index: 61
Length: 101.0, Index: 116
Length: 102.0, Index: 54
Length: 185.0, Index: 93
Length: 187.0, Index: 913
Length: 187.0, Index: 505
Length: 188.0, Index: 863
Length: 191.0, Index: 843
Length: 193.0, Index: 576
Length: 193.0, Index: 229
Length: 194.0, Index: 211
Length: 194.0, Index: 457
Length: 194.0, Index: 40


Corner detection is successful